In [ ]:
import os
import sys
import xarray as xr
import cftime
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import mplotutils as mpu


In [ ]:
dir_trendy_v8_S2  = '/Trendy/Data/SLAND_Trendy-v8_S2_LatLon/'
dir_trendy_v8_S3  = '/Trendy/Data/SLAND_Trendy-v8_S3_LatLon/'
dir_trendy_v9_S2  = '/Trendy/Data/SLAND_Trendy-v9_S2_LatLon/'
dir_trendy_v9_S3  = '/Trendy/Data/SLAND_Trendy-v9_S3_LatLon/'
dir_trendy_v10_S2 = '/Trendy/Data/SLAND_Trendy-v10_S2_LatLon/'
dir_trendy_v10_S3 = '/Trendy/Data/SLAND_Trendy-v10_S3_LatLon/'
dir_forest        = '/GCB2021_commentary/Data/forest_masks/'
dir_fig           = '/GCB2021_commentary/Figures/'


## Calculate forest fraction weight from Hansen 2013 compared to pre-industrial Trendy data

In [ ]:
#Define models
models_all = ['CABLE-POP', 'CLASSIC', 'CLM5.0', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.1', 'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs']

#Select all natural land or forests only
selection = 'NaturalLandCoverFraction'
selection = 'ForestFraction'


#Skip CABLE-POP as there is a problem with the natural land cover fraction (I did not investigate this further, since it only affects a Supplementary Figure)
if selection=='NaturalLandCoverFraction':
    models = models_all[1::]
else:
    models = models_all

#Define maximum values for weighting factor
max_values = [2, 10, 50, 100]

if selection=='ForestFraction':
    var_name     = 'forest_fraction'
    var_name_out = 'w_FF'
    fname_out = 'Weights_ForestFraction'
elif selection=='NaturalLandCoverFraction':
    var_name     = 'natural_land_cover_fraction'
    var_name_out = 'w_NLCF'
    fname_out = 'Weights_NaturalLandCoverFraction'

#Loop over files/models
create = 1
area_coll = []
for ii, model in enumerate(models):
    
    
    #Define correct Trendy folder
    if model=='CABLE-POP':
        dir_trendy_S2 = dir_trendy_v8_S2
        dir_trendy_S3 = dir_trendy_v8_S3
    elif model=='CLM5.0':
        dir_trendy_S2 = dir_trendy_v9_S2
        dir_trendy_S3 = dir_trendy_v8_S3
    elif model=='CLASSIC-N':
        dir_trendy_S2 = dir_trendy_v10_S2
        dir_trendy_S3 = dir_trendy_v10_S3
    else:
        dir_trendy_S2 = dir_trendy_v9_S2
        dir_trendy_S3 = dir_trendy_v9_S3

    #Use forest fraction of JULES-ES-1.0 for JULES-ES-1.1 (since it is not available for the latter)
    if model=='JULES-ES-1.1':  model_read = 'JULES-ES-1.0'
    else:                      model_read = model
        
    #Get files that contain forest fraction for Trendy models
    files_S2 = sorted([file for file in os.listdir(dir_trendy_S2) if model_read + '_' in file and selection in file and 'regrid360x720-ForestMask' in file])
    files_S3 = sorted([file for file in os.listdir(dir_trendy_S3) if model_read + '_' in file and selection in file and 'regrid360x720-ForestMask' in file])
    if len(files_S2)!=1:  sys.exit('File name not unique')
    if len(files_S3)!=1:  sys.exit('File name not unique')
    
    #Read data
    data_S2 = xr.open_dataset(dir_trendy_S2 + files_S2[0])
    data_S3 = xr.open_dataset(dir_trendy_S3 + files_S3[0])
    
    data_S2 = data_S2[var_name].to_dataset(name=var_name)
    data_S3 = data_S3[var_name].to_dataset(name=var_name)
    
    #Calculate average over last 20 years if time variable exists
    if 'time' in data_S2.dims:  data_S2 = data_S2.isel(time=slice(-20, None)).mean('time')
    if 'time' in data_S3.dims:  data_S3 = data_S3.isel(time=slice(-20, None)).mean('time')        
   
    #Collect data in one dataset
    if create==1:
        data_for_PI_S2 = data_S2
        data_for_PI_S3 = data_S3
        create = 0
    else:
        data_for_PI_S2 = xr.concat((data_for_PI_S2, data_S2), dim='model')
        data_for_PI_S3 = xr.concat((data_for_PI_S3, data_S3), dim='model')

#Add model names
data_for_PI_S2['model'] = models
data_for_PI_S3['model'] = models

#Calculate fraction as fraction of S3 to S2 simulations
for_weight = data_for_PI_S3[var_name] / data_for_PI_S2[var_name]

#Only allow weighting factors larger than 1 for land grid points with at least 0.1% forest cover
for_weight = xr.where(data_for_PI_S2[var_name]>0.001, for_weight, 1)
# for_weight = xr.where(data_for_PI_S3[var_name]>0.001, for_weight, 0)

#Replace NaNs bei 0
for_weight = for_weight.fillna(0)

#Convert to data set
for_weight = for_weight.to_dataset(name=var_name_out)

#Save in file
fname_weight = dir_forest + fname_out + '_DGVMs-S2-S3_regrid360x720-ForestMask.nc'
if os.path.exists(fname_weight):  os.remove(fname_weight)
for_weight.to_netcdf(fname_weight)
